## ML-uppgift

### Vi börjar med att importera bibliotek och läsa in filen. 

In [45]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.linear_model import Lasso, ElasticNet
from sklearn.ensemble import HistGradientBoostingRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import mean_squared_error

In [43]:

df = pd.read_csv('car_price_dataset.csv', sep=";")  


I mitt experimenterande visade det sig att separatorn i CSV-filen är ; och inte , - fixade det ovan.

Hur ser dataunderlaget ut?

In [22]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 10 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Brand         10000 non-null  object 
 1   Model         10000 non-null  object 
 2   Year          10000 non-null  int64  
 3   Engine_Size   10000 non-null  float64
 4   Fuel_Type     10000 non-null  object 
 5   Transmission  10000 non-null  object 
 6   Mileage       10000 non-null  int64  
 7   Doors         10000 non-null  int64  
 8   Owner_Count   10000 non-null  int64  
 9   Price         10000 non-null  int64  
dtypes: float64(1), int64(5), object(4)
memory usage: 781.4+ KB


In [23]:
print(df.isnull().sum())

Brand           0
Model           0
Year            0
Engine_Size     0
Fuel_Type       0
Transmission    0
Mileage         0
Doors           0
Owner_Count     0
Price           0
dtype: int64


Inga null-värden

In [24]:
print(df.describe())
print(df.select_dtypes(include='object').nunique())


               Year   Engine_Size        Mileage         Doors   Owner_Count  \
count  10000.000000  10000.000000   10000.000000  10000.000000  10000.000000   
mean    2011.543700      3.000560  149239.111800      3.497100      2.991100   
std        6.897699      1.149324   86322.348957      1.110097      1.422682   
min     2000.000000      1.000000      25.000000      2.000000      1.000000   
25%     2006.000000      2.000000   74649.250000      3.000000      2.000000   
50%     2012.000000      3.000000  149587.000000      3.000000      3.000000   
75%     2017.000000      4.000000  223577.500000      4.000000      4.000000   
max     2023.000000      5.000000  299947.000000      5.000000      5.000000   

             Price  
count  10000.00000  
mean    8852.96440  
std     3112.59681  
min     2000.00000  
25%     6646.00000  
50%     8858.50000  
75%    11086.50000  
max    18301.00000  
Brand           10
Model           30
Fuel_Type        4
Transmission     3
dtype: int64


Alla rader är unika, inga dubletter och inga värden saknas. Det finns tio bilmärken, 30 olika modeller, fyra bränsletyper och tre växellådor. Hur kan det finnas tre olika växellådor frågar sig vän av ordning? Vilka är det? Vi kollar.

In [25]:
print(df.Transmission.unique())

['Manual' 'Automatic' 'Semi-Automatic']


Frågan är hur semi-automatic och automatic skiljer sig? Kanske ska man slå ihop dessa till en så man får manuell eller automatisk växellåda? Vi kollar om det där semi-automatic kanske bara är brus som förvirrar modellen?


In [26]:
trans_pct = df["Transmission"].value_counts(normalize=True) * 100
print(trans_pct.round(1))


Transmission
Manual            33.7
Automatic         33.2
Semi-Automatic    33.1
Name: proportion, dtype: float64


Nej, det är en tredjedel av varje. Märkligt, men jag lämnar detta beslut tills jag ser hur priset korrelerar med de olika alternativen.

Först gör vi dock en enkel koll på de numeriska kolumnerna och hur de korrelererar med priset. 

In [27]:
numeric_cols = ["Year", "Engine_Size", "Mileage", "Doors", "Owner_Count", "Price"]
print(df[numeric_cols].corr()["Price"].sort_values(ascending=False))


Price          1.000000
Year           0.663036
Engine_Size    0.357403
Owner_Count    0.002656
Doors          0.000511
Mileage       -0.551227
Name: Price, dtype: float64


Och så de textbaserade kolumnerna. Vi stryker Brand ur hanteringen eftersom det ju implicit bestäms av Model

In [35]:
cats = ["Model", "Fuel_Type", "Transmission"]

# 1. Dummies
X_cat = pd.get_dummies(df[cats], drop_first=True)

# 2. Korrelationer för alla dummies mot Price
corrs = X_cat.join(df["Price"]).corr()["Price"]

# 3. Summera absoluta korrelationer per variabel
model_corr        = corrs[corrs.index.str.startswith("Model_")].abs().sum()
fuel_type_corr    = corrs[corrs.index.str.startswith("Fuel_Type_")].abs().sum()
transmission_corr = corrs[corrs.index.str.startswith("Transmission_")].abs().sum()

summary = pd.Series(
    {
        "Model": model_corr,
        "Fuel_Type": fuel_type_corr,
        "Transmission": transmission_corr,
    },
    name="Total_abs_corr_with_Price"
)

print(summary)


Model           0.219170
Fuel_Type       0.415330
Transmission    0.245259
Name: Total_abs_corr_with_Price, dtype: float64


Nu blir jag ändå lite misstänksam mot växellådorna där. Jag får kolla lite mer noga. 

In [30]:
print(df.groupby("Transmission")["Price"].describe())

med_iqr = df.groupby("Transmission")["Price"].agg(
    median="median",
    q1=lambda s: s.quantile(0.25),
    q3=lambda s: s.quantile(0.75)
)
print(med_iqr)


                 count         mean          std     min     25%      50%  \
Transmission                                                                
Automatic       3317.0  9938.252939  3025.692193  2000.0  7784.0  10034.0   
Manual          3372.0  8363.426157  3019.758983  2000.0  6159.5   8341.5   
Semi-Automatic  3311.0  8264.266385  3006.190812  2000.0  6069.0   8263.0   

                    75%      max  
Transmission                      
Automatic       12070.0  18301.0  
Manual          10514.5  16844.0  
Semi-Automatic  10364.5  17078.0  
                 median      q1       q3
Transmission                            
Automatic       10034.0  7784.0  12070.0
Manual           8341.5  6159.5  10514.5
Semi-Automatic   8263.0  6069.0  10364.5


Ajdå - semi-automatic är alltså en informativ feature ändå. Ja, ja - då får den vara kvar. 

## Dags att förbereda för träning


Det första vi ska göra är att ta itu med Doors och Owner_Count. Tillför dessa något eller är de mest brus i vårt dataset? 

Vi kör ett feature-set med alla variabler (fortfarande utan brand som vi ju valt bort som implicit bestämt av Model), ett utan doors och owner count, samt ett utan doors respektive owner count. 

In [37]:
base_cols = ["Model", "Year", "Engine_Size", "Fuel_Type",
             "Transmission", "Mileage"]

# Alla variabler
cols_all = base_cols + ["Doors", "Owner_Count"]

# Utan båda
cols_none = base_cols

# Utan Doors (men med Owner_Count)
cols_no_doors = base_cols + ["Owner_Count"]

# Utan Owner_Count (men med Doors)
cols_no_owner = base_cols + ["Doors"]


Vi gör en test med Lasso (endast i syfte att granska relevansen för de två kolumnerna).


In [ ]:
def make_X(df, cols):
    X = df[cols]
    return pd.get_dummies(
        X,
        columns=["Model", "Fuel_Type", "Transmission"],
        drop_first=True
    )

def rmse_lasso(X_enc, y):
    X_train, X_test, y_train, y_test = train_test_split(
        X_enc, y, test_size=0.2, random_state=42
    )
    model = Lasso(alpha=0.1, max_iter=10000)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    return np.sqrt(mean_squared_error(y_test, y_pred))


Och så ska vi se hur RMSE påverkas.

In [41]:
y = df["Price"]

rmse_all      = rmse_lasso(make_X(df, cols_all), y)
rmse_none     = rmse_lasso(make_X(df, cols_none), y)
rmse_no_doors = rmse_lasso(make_X(df, cols_no_doors), y)
rmse_no_owner = rmse_lasso(make_X(df, cols_no_owner), y)

print("RMSE alla features:        ", rmse_all)
print("RMSE utan Doors/Owner:     ", rmse_none)
print("RMSE utan Doors (med Owner)", rmse_no_doors)
print("RMSE utan Owner (med Doors)", rmse_no_owner)


RMSE alla features:         64.73418582536543
RMSE utan Doors/Owner:      64.7130558287325
RMSE utan Doors (med Owner) 64.71547309807511
RMSE utan Owner (med Doors) 64.73198122735877


Det visar alltså att RMSE inte påverkas mer än i liten utsträckning av Doors/Owner, varför dessa slopas i genomgången av modeller.

## Dags att testa modeller

Vi har valt att dela upp modellerna så att alla får arbeta med både linjär och trädbaserad modell. Här följer genomgång av Lasso, ElasticNet och HistGradientBoostingRegressor. Om tid finns följer även KNeighborsRegressor. 

En gemensam feature-set för alla modeller.

In [50]:
features = ["Model", "Year", "Engine_Size", "Fuel_Type",
            "Transmission", "Mileage"]
X = df[features]
y = df["Price"]

X_enc = pd.get_dummies(
    X,
    columns=["Model", "Fuel_Type", "Transmission"],
    drop_first=True
)


Så splittar vi upp data för träning, validering och test. Jag sparar 20 % för testningen för att få rimlig utvärdering av modellerna.

In [51]:
X_trainval, X_test, y_trainval, y_test = train_test_split(
    X_enc, y, test_size=0.2, random_state=42
)


# Lasso



In [80]:
lasso_pipe = Pipeline([
    ("scaler", StandardScaler(with_mean=False)),  
    ("model", Lasso(max_iter=10000))
])

# Hyperparametrar 
lasso_param_grid = {
    "model__alpha": [0.01, 0.1, 1.0]
}

# GridSearchCV 
lasso_grid = GridSearchCV(
    lasso_pipe,
    param_grid=lasso_param_grid,
    scoring="neg_root_mean_squared_error",
    cv=5,
    n_jobs=-1
)

lasso_grid.fit(X_trainval, y_trainval)

print("Bästa alpha:", lasso_grid.best_params_)
print("CV-RMSE:", -lasso_grid.best_score_)


Bästa alpha: {'model__alpha': 1.0}
CV-RMSE: 85.59669801400972


Alpha ska alltså vara 1.0 och RMSE för min korsvalidering är 89. 

In [53]:
best_lasso = lasso_grid.best_estimator_

y_pred_test = best_lasso.predict(X_test)
lasso_rmse_test = np.sqrt(mean_squared_error(y_test, y_pred_test))

print("Test-RMSE Lasso:", lasso_rmse_test)


Test-RMSE Lasso: 64.89775587865122


Märkligt nog är RMSE för mina 20% testdata betydligt lägre - jag får dubbelkolla detta.

In [54]:
y_pred_trainval = best_lasso.predict(X_trainval)
rmse_trainval = np.sqrt(mean_squared_error(y_trainval, y_pred_trainval))
print("Train/val-RMSE (med best_lasso):", rmse_trainval)

Train/val-RMSE (med best_lasso): 90.25184605693242


Hmm. Skumt. Kan randomiseringen ha blivit så skum? Jag får köra en extra omgång med annat random-värde. 

För att inte störa hanteringen kommenterar jag bort detta nu efter testen. Resultaten är inlagda i kodrutan.

In [63]:
""" X_trainval, X_test, y_trainval, y_test = train_test_split(
    X_enc, y, test_size=0.2, random_state=123  
)

lasso_pipe = Pipeline([
    ("scaler", StandardScaler(with_mean=False)),  
    ("model", Lasso(max_iter=10000))
])

# Hyperparametrar 
lasso_param_grid = {
    "model__alpha": [0.01, 0.1, 1.0, 10]
}

# GridSearchCV 
lasso_grid = GridSearchCV(
    lasso_pipe,
    param_grid=lasso_param_grid,
    scoring="neg_root_mean_squared_error",
    cv=5,
    n_jobs=-1
)

lasso_grid.fit(X_trainval, y_trainval)

print("Bästa alpha:", lasso_grid.best_params_)
print("CV-RMSE:", -lasso_grid.best_score_)

best_lasso = lasso_grid.best_estimator_

y_pred_test = best_lasso.predict(X_test)
lasso_rmse_test = np.sqrt(mean_squared_error(y_test, y_pred_test))

print("Test-RMSE Lasso:", lasso_rmse_test)

"""

# Resultat
# Bästa alpha: {'model__alpha': 1.0}
# CV-RMSE: 85.59669801400972
# Test-RMSE Lasso: 70.96541649575882

' X_trainval, X_test, y_trainval, y_test = train_test_split(\n    X_enc, y, test_size=0.2, random_state=123  \n)\n\nlasso_pipe = Pipeline([\n    ("scaler", StandardScaler(with_mean=False)),  \n    ("model", Lasso(max_iter=10000))\n])\n\n# Hyperparametrar \nlasso_param_grid = {\n    "model__alpha": [0.01, 0.1, 1.0, 10]\n}\n\n# GridSearchCV \nlasso_grid = GridSearchCV(\n    lasso_pipe,\n    param_grid=lasso_param_grid,\n    scoring="neg_root_mean_squared_error",\n    cv=5,\n    n_jobs=-1\n)\n\nlasso_grid.fit(X_trainval, y_trainval)\n\nprint("Bästa alpha:", lasso_grid.best_params_)\nprint("CV-RMSE:", -lasso_grid.best_score_)\n\nbest_lasso = lasso_grid.best_estimator_\n\ny_pred_test = best_lasso.predict(X_test)\nlasso_rmse_test = np.sqrt(mean_squared_error(y_test, y_pred_test))\n\nprint("Test-RMSE Lasso:", lasso_rmse_test)\n\n'

Fortfarande en skum skillnad. Har mina data extremvärden som stör? 

In [65]:
# df[["Price", "Year", "Engine_Size", "Mileage"]].describe()

# Data visas som tabell och gick inte att klippa in här. Kör koden vid intresse.

Nej, det ser inte ut att finnas några extremt avvikande värden. 1,5×IQR‑regeln antyder att data är relativt jämnt fördelade i rimliga intervall. Skillnaden mellan CV‑RMSE och test‑RMSE beror därför troligen på en naturlig variation mellan i det här datasetet

# Elasticnet


In [66]:
from sklearn.linear_model import ElasticNet

elastic_pipe = Pipeline([
    ("scaler", StandardScaler(with_mean=False)),
    ("model", ElasticNet(max_iter=10000))
])

elastic_param_grid = {
    "model__alpha": [0.01, 0.1, 1.0, 10],
    "model__l1_ratio": [0.2, 0.5, 0.8]  
}

elastic_grid = GridSearchCV(
    elastic_pipe,
    param_grid=elastic_param_grid,
    scoring="neg_root_mean_squared_error",
    cv=5,
    n_jobs=-1
)

elastic_grid.fit(X_trainval, y_trainval)

print("Bästa hyperparametrar:", elastic_grid.best_params_)
print("CV-RMSE ElasticNet:", -elastic_grid.best_score_)

best_elastic = elastic_grid.best_estimator_

y_pred_test_elastic = best_elastic.predict(X_test)
elastic_rmse_test = np.sqrt(mean_squared_error(y_test, y_pred_test_elastic))
print("Test-RMSE ElasticNet:", elastic_rmse_test)


Bästa hyperparametrar: {'model__alpha': 0.01, 'model__l1_ratio': 0.8}
CV-RMSE ElasticNet: 86.24901822235338
Test-RMSE ElasticNet: 71.56560996636526


Även ElasticNet har ett tydligt högre CV‑RMSE än test‑RMSE. Gissningvis är det korsvalideringen som ger en mer försiktig uppskattning av modellens fel medan de 20% testdata är lättare att förutsäga. Modellen är inte överanpassad till testdatan, snarare är CV‑värdet konservativt.

# HistGradientBoostingRegressor


In [69]:
hgb_pipe = Pipeline([
    ("scaler", StandardScaler(with_mean=False)),  
    ("model", HistGradientBoostingRegressor(random_state=123))
])

hgb_param_grid = {
    "model__learning_rate": [0.05, 0.1, 0.2], 
    "model__max_depth": [None, 3, 5], 
    "model__max_iter": [200, 300, 500],
    "model__min_samples_leaf": [20, 50, 100]
}

hgb_grid = GridSearchCV(
    hgb_pipe,
    param_grid=hgb_param_grid,
    scoring="neg_root_mean_squared_error",
    cv=5,
    n_jobs=-1
)

hgb_grid.fit(X_trainval, y_trainval)

print("Bästa hyperparametrar:", hgb_grid.best_params_)
print("CV-RMSE HGB:", -hgb_grid.best_score_)





Bästa hyperparametrar: {'model__learning_rate': 0.1, 'model__max_depth': 3, 'model__max_iter': 500, 'model__min_samples_leaf': 20}
CV-RMSE HGB: 136.56778437983817


Och så testar vi. 

In [70]:
best_hgb = hgb_grid.best_estimator_

y_pred_test_hgb = best_hgb.predict(X_test)
hgb_rmse_test = np.sqrt(mean_squared_error(y_test, y_pred_test_hgb))
print("Test-RMSE HGB:", hgb_rmse_test)


Test-RMSE HGB: 135.4886323023571


Här stämmer siffrorna väl med varandra, men med ett RMSE på 135 är det tydligt att det här *inte* är rätt modell. 

# KNeighborsRegressor

In [74]:

knn_pipe = Pipeline([
    ("scaler", StandardScaler(with_mean=False)),  
    ("model", KNeighborsRegressor())
])

knn_param_grid = {
    "model__n_neighbors": [5, 10, 20, 40, 80],
    "model__weights": ["uniform", "distance"],
    "model__p": [1, 2]  
}

knn_grid = GridSearchCV(
    knn_pipe,
    param_grid=knn_param_grid,
    scoring="neg_root_mean_squared_error",
    cv=5,
    n_jobs=-1
)

knn_grid.fit(X_trainval, y_trainval)

print("Bästa hyperparametrar:", knn_grid.best_params_)
print("CV-RMSE KNN:", -knn_grid.best_score_)



Bästa hyperparametrar: {'model__n_neighbors': 5, 'model__p': 1, 'model__weights': 'distance'}
CV-RMSE KNN: 1338.7537392729776


Sjukt dålig modell för de här data, men vi verifierar ändå mot testdata.

In [75]:
best_knn = knn_grid.best_estimator_

y_pred_test_knn = best_knn.predict(X_test)
knn_rmse_test = np.sqrt(mean_squared_error(y_test, y_pred_test_knn))
print("Test-RMSE KNN:", knn_rmse_test)


Test-RMSE KNN: 1227.9032409509753


# Resultat
| Modell                          | CV‑RMSE | Test‑RMSE |
|---------------------------------|--------:|----------:|
| Lasso                           | 85      | 65        |
| ElasticNet                      | 86      | 72        |
| HistGradientBoostingRegressor   | 137     | 135       |
| KNeighborsRegressor             | 1339    | 1228      |

Utifrån RMSE‑resultaten verkar de linjära modellerna (Lasso och ElasticNet) vara klart mest lämpliga för det här datasetet, medan trädmodellerna HistGradientBoostingRegressor och KNeighborsRegressor ger betydligt större fel trots att jag suttit och försökt finslipa hyperparametrarna.

# Bonus
Jag provar ett par modeller till innan jag lägger av. 


# Ridge


In [76]:
from sklearn.linear_model import Ridge

ridge_pipe = Pipeline([
    ("scaler", StandardScaler(with_mean=False)),
    ("model", Ridge(max_iter=10000))
])

ridge_param_grid = {
    "model__alpha": [0.01, 0.1, 1.0, 10, 100]
}

ridge_grid = GridSearchCV(
    ridge_pipe,
    param_grid=ridge_param_grid,
    scoring="neg_root_mean_squared_error",
    cv=5,
    n_jobs=-1
)

ridge_grid.fit(X_trainval, y_trainval)

print("Bästa hyperparametrar Ridge:", ridge_grid.best_params_)
print("CV-RMSE Ridge:", -ridge_grid.best_score_)

best_ridge = ridge_grid.best_estimator_

y_pred_test_ridge = best_ridge.predict(X_test)
ridge_rmse_test = np.sqrt(mean_squared_error(y_test, y_pred_test_ridge))
print("Test-RMSE Ridge:", ridge_rmse_test)


Bästa hyperparametrar Ridge: {'model__alpha': 0.01}
CV-RMSE Ridge: 85.65859113366761
Test-RMSE Ridge: 70.87104847861852


# RandomForestRegressor

In [77]:
from sklearn.ensemble import RandomForestRegressor

rf_pipe = Pipeline([
    ("scaler", StandardScaler(with_mean=False)),  
    ("model", RandomForestRegressor(random_state=123))
])

rf_param_grid = {
    "model__n_estimators": [100, 300],
    "model__max_depth": [None, 10, 20],
    "model__min_samples_split": [2, 5, 10],
    "model__min_samples_leaf": [1, 2, 4]
}

rf_grid = GridSearchCV(
    rf_pipe,
    param_grid=rf_param_grid,
    scoring="neg_root_mean_squared_error",
    cv=5,
    n_jobs=-1
)

rf_grid.fit(X_trainval, y_trainval)

print("Bästa hyperparametrar RF:", rf_grid.best_params_)
print("CV-RMSE RF:", -rf_grid.best_score_)

best_rf = rf_grid.best_estimator_

y_pred_test_rf = best_rf.predict(X_test)
rf_rmse_test = np.sqrt(mean_squared_error(y_test, y_pred_test_rf))
print("Test-RMSE RF:", rf_rmse_test)


Bästa hyperparametrar RF: {'model__max_depth': None, 'model__min_samples_leaf': 2, 'model__min_samples_split': 2, 'model__n_estimators': 300}
CV-RMSE RF: 515.4032836524248
Test-RMSE RF: 467.30628916472546


Ridge gav ungefär samma resultat som de andra två linjära modellerna. RandomForestRegressor tog en bra stund så jag fick förhoppningar, men de kom på skam. :(